In [1]:
! pip install pyspark

In [2]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('RentSpree').getOrCreate()

your 131072x1 screen size is bogus. expect trouble


23/03/21 11:59:30 WARN Utils: Your hostname, LAPTOP-E01KR4V4 resolves to a loopback address: 127.0.1.1; using 172.19.203.29 instead (on interface eth0)
23/03/21 11:59:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/21 11:59:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
## read the dataset
df=spark.read.option('header','true').csv('daily_transactions/daily_transactions.csv',inferSchema=True)

In [4]:
### Check the schema
df.printSchema()

root
 |-- _id: string (nullable = true)
 |-- event_date: timestamp (nullable = true)
 |-- usertype: string (nullable = true)



In [5]:
df.show(10)

+--------------------+-------------------+--------+
|                 _id|         event_date|usertype|
+--------------------+-------------------+--------+
|By1+rEy20nW/sgReh...|2021-02-10 00:00:00|       A|
|x7lxSW+y6Kymzxf1y...|2021-02-10 00:00:00|       A|
|BQ7feJAnc6ntg4PaO...|2021-02-24 00:00:00|       A|
|xmc9CsW3Q7K9HK5+p...|2020-12-08 00:00:00|       A|
|Dt0y6YCKnC/mD9Sm1...|2020-06-17 00:00:00|       A|
|8VN+wWDZyD+/H7IRI...|2021-01-07 00:00:00|       A|
|WXsKNZ5l/rqxIk3pw...|2020-12-18 00:00:00|       A|
|vb6FkuOhgdOzmGpgl...|2021-01-18 00:00:00|       A|
|hginiEQhCS1lA6bns...|2020-05-02 00:00:00|       A|
|TqcP/Vc8nfJIaOG9T...|2020-11-08 00:00:00|       A|
+--------------------+-------------------+--------+
only showing top 10 rows



## Remove Id with more than one usertype

In [6]:
df.tail(5)

[Row(_id='1pjAQ3L6g2nDMPXdEs+T16fUjZ8=', event_date=datetime.datetime(2020, 4, 21, 0, 0), usertype='D'),
 Row(_id='z/EFPSvbktIGuiH61IOKnZ3JgTY=', event_date=datetime.datetime(2021, 4, 29, 0, 0), usertype='D'),
 Row(_id='7Z4YJCwAye7Al3+ZVe7ya5uymPQ=', event_date=datetime.datetime(2020, 7, 29, 0, 0), usertype='D'),
 Row(_id='ostGCvBSK4ym3oeHL6FEDxUUsOk=', event_date=datetime.datetime(2019, 11, 18, 0, 0), usertype='D'),
 Row(_id='ostGCvBSK4ym3oeHL6FEDxUUsOk=', event_date=datetime.datetime(2019, 11, 18, 0, 0), usertype='A')]

In [7]:
dis_df = df.select(['_id','usertype']).distinct().groupBy(['_id']).count()
duplicate_id_lst = dis_df.select('_id').filter(dis_df['count']>1).rdd.map(lambda x: x._id).collect()
df = df[~df['_id'].isin(duplicate_id_lst)]
df.tail(5)

[Row(_id='Ughw2F3IDh5eu0XCsZe2VHOHvrk=', event_date=datetime.datetime(2021, 1, 10, 0, 0), usertype='D'),
 Row(_id='DnlHKHuPAkgzd0sfGuTien15E3o=', event_date=datetime.datetime(2020, 7, 14, 0, 0), usertype='D'),
 Row(_id='1pjAQ3L6g2nDMPXdEs+T16fUjZ8=', event_date=datetime.datetime(2020, 4, 21, 0, 0), usertype='D'),
 Row(_id='z/EFPSvbktIGuiH61IOKnZ3JgTY=', event_date=datetime.datetime(2021, 4, 29, 0, 0), usertype='D'),
 Row(_id='7Z4YJCwAye7Al3+ZVe7ya5uymPQ=', event_date=datetime.datetime(2020, 7, 29, 0, 0), usertype='D')]

## Get new users

In [8]:
import pyspark.sql.functions as F

first_event_df = df.groupBy('_id').agg(F.min('event_date').alias("first_event"))
join_df = df.join(first_event_df, df['_id']==first_event_df['_id'], 'inner').drop(first_event_df._id)
newuser_df = join_df.withColumn("new_user", join_df["event_date"]==join_df["first_event"]).withColumn('month', F.date_format(join_df["event_date"], "MMMM yyyy"))
newuser_count_df = newuser_df.filter(newuser_df["new_user"]==True).groupBy("month").count().withColumnRenamed("count","new_users")

newuser_count_df.sort(newuser_count_df['new_users'], ascending=False).show()

+--------------+---------+
|         month|new_users|
+--------------+---------+
|    April 2021|    34927|
|    March 2021|    31369|
|     July 2020|    30152|
|   August 2020|    28019|
|  January 2021|    23389|
|     June 2020|    22949|
|  October 2020|    22854|
| February 2021|    22676|
|September 2020|    21693|
|      May 2020|    20295|
| December 2020|    19071|
| November 2020|    18875|
|    April 2020|    15117|
|    March 2020|    13021|
|     July 2019|    12985|
| February 2020|    12891|
|  January 2020|    12732|
|     June 2019|    11833|
|   August 2019|    11755|
|  October 2019|    10318|
+--------------+---------+
only showing top 20 rows



## Get returning users

In [9]:
data = [("A",360),
        ("B",360),
        ("C",120),
        ("D",260)]
churn_period_df = spark.createDataFrame(data=data,schema=['usertype', 'period'])
churn_period_df.show()

+--------+------+
|usertype|period|
+--------+------+
|       A|   360|
|       B|   360|
|       C|   120|
|       D|   260|
+--------+------+



In [10]:
from pyspark.sql.window import Window

windowSpec  = Window.partitionBy("_id").orderBy("event_date")
previous_event_df = df.withColumn("previous_event", F.lag("event_date",1).over(windowSpec)).na.drop()
date_diff_df = previous_event_df.withColumn('date_diff', F.datediff(previous_event_df['event_date'], previous_event_df['previous_event']))
join_period_df = date_diff_df.join(churn_period_df, date_diff_df['usertype']==churn_period_df['usertype'], 'inner').drop(churn_period_df['usertype'])
return_user_df = join_period_df.withColumn("return_user", join_period_df["date_diff"]>=join_period_df["period"]).withColumn('month', F.date_format(df["event_date"], "MMMM yyyy"))
return_count_df = return_user_df.select(['month', '_id']).distinct().filter(return_user_df["return_user"]==True).groupBy("month").count().withColumnRenamed("count","returning_users")

return_count_df.sort(return_count_df["returning_users"], ascending=False).show()

+--------------+---------------+
|         month|returning_users|
+--------------+---------------+
|    April 2021|            946|
|    March 2021|            719|
| February 2021|            569|
|  January 2021|            560|
|  October 2020|            393|
| December 2020|            387|
| November 2020|            351|
|   August 2020|            347|
|     July 2020|            345|
|September 2020|            289|
|     June 2020|            210|
|      May 2020|            157|
|      May 2021|             96|
|    March 2020|             94|
|    April 2020|             86|
| February 2020|             81|
|  January 2020|             76|
| December 2019|             41|
| November 2019|             22|
|  October 2019|             15|
+--------------+---------------+
only showing top 20 rows



## Get dropoff users

In [11]:
windowSpec = Window.partitionBy("_id").orderBy("event_date")
max_date = df.select(F.max('event_date'))
next_event_df = df.withColumn("next_event", F.lead("event_date",1).over(windowSpec)).crossJoin(max_date).withColumnRenamed('max(event_date)', 'max_date')
next_event_df = next_event_df.select(['_id','event_date','usertype',F.coalesce(next_event_df['next_event'], next_event_df['max_date']).alias('next_event')])
date_diff_df = next_event_df.withColumn('date_diff', F.datediff(next_event_df['next_event'], next_event_df['event_date']))
join_period_df = date_diff_df.join(churn_period_df, date_diff_df['usertype']==churn_period_df['usertype'], 'inner').drop(churn_period_df['usertype'])
dropped_user_df = join_period_df.withColumn("dropped", join_period_df["date_diff"]>=join_period_df["period"]).withColumn("period", join_period_df.period.cast('int'))
dropped_dt_df = dropped_user_df.withColumn("dropped_date", F.date_add(dropped_user_df['event_date'], dropped_user_df['period'])).filter(dropped_user_df["dropped"]==True)
dropped_mn_df = dropped_dt_df.withColumn('month', F.date_format(dropped_dt_df["dropped_date"], "MMMM yyyy"))
dropped_count_df = dropped_mn_df.select(['month', '_id', 'dropped']).distinct().groupBy("month").count().withColumnRenamed("count","dropoff_users")

dropped_count_df.sort(dropped_count_df['dropoff_users']).show()

+--------------+-------------+
|         month|dropoff_users|
+--------------+-------------+
|September 2019|          269|
|  October 2019|          516|
| December 2019|          534|
| November 2019|          550|
|    April 2020|          828|
|    March 2020|          964|
| February 2020|         1025|
|  January 2020|         1056|
|      May 2021|         2433|
|      May 2020|         8111|
| December 2020|         9399|
| November 2020|         9846|
|  October 2020|        10238|
|September 2020|        10312|
|     June 2020|        10923|
|   August 2020|        11222|
|    March 2021|        12231|
|     July 2020|        12456|
| February 2021|        12618|
|  January 2021|        13126|
+--------------+-------------+
only showing top 20 rows



## Join everything

In [12]:
result_df = dropped_count_df.join(return_count_df, dropped_count_df['month']==return_count_df['month'], 'outer').withColumn('c_month', F.coalesce(dropped_count_df['month'], return_count_df['month'])).drop('month')
result_df = result_df.join(newuser_count_df, result_df['c_month']==newuser_count_df['month'], 'outer').withColumn('c_month', F.coalesce(result_df['c_month'], newuser_count_df['month'])).drop('month')
result_df = result_df.withColumnRenamed('c_month', 'month').na.fill(0).select(['month', 'dropoff_users', 'returning_users', 'new_users'])

result_df.show()

23/03/21 11:59:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/21 11:59:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+-------------+-------------+---------------+---------+
|        month|dropoff_users|returning_users|new_users|
+-------------+-------------+---------------+---------+
|   April 2020|          828|             86|    15117|
|   April 2021|        15101|            946|    34927|
|  August 2019|            0|              0|    11755|
|  August 2020|        11222|            347|    28019|
|December 2019|          534|             41|     8567|
|December 2020|         9399|            387|    19071|
|February 2020|         1025|             81|    12891|
|February 2021|        12618|            569|    22676|
| January 2020|         1056|             76|    12732|
| January 2021|        13126|            560|    23389|
|    July 2019|            0|              0|    12985|
|    July 2020|        12456|            345|    30152|
|    June 2019|            0|              0|    11833|
|    June 2020|        10923|            210|    22949|
|   March 2020|          964|             94|   

In [13]:
from datetime import datetime

from_date = 'April 2020'
to_date = 'April 2021'

from_date = datetime.strptime(from_date, "%B %Y").date()
to_date = datetime.strptime(to_date, "%B %Y").date()

result_df = result_df.withColumn('month_num',F.to_date(result_df['month'],"MMMM yyyy"))
result_df = result_df.filter(F.col('month_num').between(from_date, to_date)).sort(result_df['month_num']).drop('month_num')
result_df.show()

23/03/21 12:00:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/21 12:00:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+--------------+-------------+---------------+---------+
|         month|dropoff_users|returning_users|new_users|
+--------------+-------------+---------------+---------+
|    April 2020|          828|             86|    15117|
|      May 2020|         8111|            157|    20295|
|     June 2020|        10923|            210|    22949|
|     July 2020|        12456|            345|    30152|
|   August 2020|        11222|            347|    28019|
|September 2020|        10312|            289|    21693|
|  October 2020|        10238|            393|    22854|
| November 2020|         9846|            351|    18875|
| December 2020|         9399|            387|    19071|
|  January 2021|        13126|            560|    23389|
| February 2021|        12618|            569|    22676|
|    March 2021|        12231|            719|    31369|
|    April 2021|        15101|            946|    34927|
+--------------+-------------+---------------+---------+



In [14]:
import shutil

result_df.coalesce(1).write.csv(path="result", header=True, mode="overwrite")

file_name=list(filter(lambda x: x.endswith('.csv'), shutil.os.listdir('result')))[0]
shutil.move(f'result/{file_name}', 'result.csv')
shutil.rmtree('result', ignore_errors=True)

23/03/21 12:00:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/21 12:00:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
